<a href="https://www.kaggle.com/code/oswind/stockchat-towards-a-stock-market-assistant?scriptVersionId=236935572" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [22]:
# Prepare the notebook environment for use.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
!pip install -qU google-genai==1.7.0 chromadb==0.6.3 langchain-community langchain-text-splitters wikipedia

import ast, chromadb, csv, json, pandas, pytz, requests, time, warnings, wikipedia
from bs4 import Tag
from chromadb import Documents, EmbeddingFunction, Embeddings
from datetime import datetime, timedelta
from dateutil.parser import parse
from dateutil.tz import gettz
from google import genai
from google.api_core import retry
from google.genai import types
from IPython.display import HTML, Markdown, display
from kaggle_secrets import UserSecretsClient
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_text_splitters.html import HTMLSemanticPreservingSplitter
from langchain_text_splitters.json import RecursiveJsonSplitter
from tqdm import tqdm
from typing import Optional
from wikipedia.exceptions import DisambiguationError, PageError

In [23]:
# Prepare the gemini client for use.
# Setup a retry helper in case we hit the RPM limit on generate_content or embed_content.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503, 500})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)
genai.models.Models.embed_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.embed_content)

# Import the secret api keys.
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# Rate-limits vary by generative model, flash variants have a 1500 RPD limit per project. 
project_model_1 = "models/gemini-2.0-flash"
project_model_2 = "models/gemini-2.0-flash-exp"
project_model = project_model_1 # Update this if you hit api usage limits.

# Create the genai client.
client = genai.Client(api_key=GOOGLE_API_KEY)

# Laying the foundation with Gemini 2.0

<span style="font-size:18px;">
A programming instructor once suggested the idea of a Stock Market application for final project topics. They did this knowing good investing app UX is challenging. The idea has stuck with me since because it's true. In the past I've worked with some REST api's building toys. None of them could ever reach my expectations because of API limits. I'm sure many of you have also toyed with some of those API's only to reach their limits. I always knew the secret to great finance UX is a great AI to help out. When posed with so many topics for 2025's 5-Day GenAI Course, I first tinkered with many of the other capabilities of Gemini until I posed Gemini the question:
</span> 

In [24]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)
chat = client.chats.create(
    model=project_model, config=config_with_search, history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

Yes, I do. Here's some information about the stock market:

**What it is:**

*   The stock market is a place where investors buy and sell shares of companies, also known as stocks.
*   It's a network of exchanges, like the New York Stock Exchange (NYSE) or the Nasdaq.
*   It also includes over-the-counter (OTC) marketplaces where investors trade directly with each other.

**How it works:**

1.  **Companies issue shares:** Companies can list shares of their stock on an exchange through an initial public offering (IPO) to raise capital.
2.  **Investors buy shares:** Investors purchase these shares, providing the company with money to grow its business.
3.  **Trading:** Investors can then buy and sell these stocks among themselves.
4.  **Price negotiation:** Buyers offer a "bid" (the highest amount they're willing to pay), and sellers "ask" for a certain amount in exchange. The difference is called the bid-ask spread.
5.  **Supply and demand:** Each exchange tracks the supply and demand of stocks listed there.

**Purpose:**

*   The stock market provides a place where people can buy and sell fractional ownership in publicly traded companies.
*   It distributes control of some of the world's largest companies among many individual investors.
*   The buying and selling decisions of investors determine the value of those companies.
*   The market lets buyers and sellers negotiate prices, which aims to maximize fairness for both parties.

**Participants:**

*   Small individual stock investors
*   Larger investors like banks, insurance companies, pension funds, and hedge funds.

**Key concepts:**

*   **Equities (stocks or shares):** Represent an ownership interest in a company.
*   **Stock exchange:** A place where stockbrokers and traders can buy and sell shares, bonds, and other securities.
*   **Market capitalization:** The total value of a company's outstanding shares.
*   **Primary market:** Where financial assets are created and sold directly by the issuer.
*   **Secondary market:** Where existing financial assets are traded.

**Important Note:** The stock market can be complex, and it's important to do your research and understand the risks involved before investing.


# How much Gemini 2.0 knows

<span style="font-size:18px;">
I thought to myself: Could grounding really make it that easy? Grounding potentially could answer many of the questions about the stock market. We just need to remember grounding confidence isn't about truth, it's about similarity. I decided to limit myself to free tier in finding out.
</span>

In [25]:
# And so I asked a more challenging questions.
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

Okay, here's what I found regarding AMZN (Amazon) stock:

**Current Sentiment:**

*   **Analysts' Ratings:** The consensus among analysts is "Strong Buy."
*   **Valuation:** Some analysts believe Amazon is currently undervalued.
*   **Recent Performance:** AMZN stock has risen by 10.33% compared to the previous week, but the month change is a -5.56% fall. Over the last year, Amazon.com has shown a 2.54% increase.
*   **Market Volatility:** AMZN stock is 2.36% volatile and has a beta coefficient of 1.39.

**Price Targets & Forecasts:**

*   **12-Month Price Targets:** Based on recent analysis, the average price target is around $245.33, with a high forecast of $287.00 and a low of $195.00.
*   **Upside Potential:** The average price target represents a potential increase of around 30.70% from the current price.
*   **Forecasts for 2025:**
    *   Some forecasts predict AMZN will trade between $168.74 and $198.47 in 2025, averaging around $181.41.
    *   Another source forecasts a maximum price of $201 and a minimum of $161 for April 2025, with a predicted price of $186 at the end of the month.
*   **Long-Term Potential:** One analysis suggests the stock price could double in five years, potentially reaching the high $400s, assuming no major economic downturn.

**Factors to Consider:**

*   **Earnings:** Amazon is scheduled to release its next earnings report soon. Analysts expect revenue and profits to increase.
*   **AI Spending:** Amazon's plans to keep up with spending on artificial intelligence are being closely watched.
*   **Tariffs:** The potential impact of tariffs on Amazon's business is a concern for some analysts, as they could hurt profit margins and advertising revenue.
*   **Economic Conditions:** A global economic downturn could negatively impact Amazon's business.

**Key Points from Recent News:**

*   UBS analysts recently cut Amazon's price target, citing slower growth in gross merchandise volumes due to tariff-driven price hikes.
*   Analysts will likely seek more information about Amazon's plans to keep up with the spending pace of its artificial intelligence rivals.
*   Oppenheimer cut its target price for Amazon stock by 15% due to concerns that tariffs will likely lead to lower profit margins.
*   UPS is cutting jobs and closing buildings, partly due to lower Amazon volumes.

**In Summary:**

The outlook for AMZN stock appears generally positive, with many analysts rating it as a "Strong Buy" and predicting significant upside potential. However, factors like tariffs, AI spending, and the overall economic climate could influence its performance. It would be wise to keep an eye on the upcoming earnings report and any news regarding these factors.


<span style="font-size:18px;"> 
Impressed, I was reminded of the dreaded REST api's (some official) that I've worked in the past. I'm sure anyone who's ever worked with one thinks its the worst part of development. So I next asked Gemini to distill it's vast news knowledge.
</span>

In [26]:
response = chat.send_message(
    '''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

Here's a summary of what's happening with AMZN stock:

**Current Share Price:**

*   As of April 29, 2025, the current price of AMZN is $187.39. It decreased by -0.17% in the last 24 hours.

**Short-Term Trends:**

*   **Mixed Signals:** The stock is up 10.33% compared to the previous week, but down 5.56% for the month. Over the last year, it's up 2.54%.
*   **Downgraded:** Amazon stock was recently downgraded to a "Sell Candidate".
*   **Falling Prices:** The price has declined in 6 of the last 10 days.
*   **Volatility:** AMZN stock is 2.36% volatile.
*   **Potential Downturn:** Given the current short-term trend, the stock is expected to fall -25.57% during the next 3 months, potentially reaching between $119.83 and $139.48.
*   **Technical Signals:** Some negative signals have been issued that may influence short-term development. The short-term Moving Average indicates a buy signal, but the long-term average indicates a sell signal, giving a more negative forecast.

**Bullish Predictions:**

*   **Analyst Ratings:** The consensus among analysts is "Strong Buy."
*   **Upside Potential:** The average analyst price target is around $249.82, suggesting a potential increase of over 33% from the current price. Some analysts have a high estimate of $290.
*   **Long-Term Growth:** In a bullish scenario, Amazon could grow earnings roughly 20% annually over the next five years, potentially leading to a price in the high-$400s.
*   **Key Factors:** Positive developments in Amazon Web Services (AWS), a rebound in consumer spending, and enhanced operational efficiency could drive the stock higher.
*   **Revenue and Earnings Growth:** Analysts expect revenue and profits to increase.
*   **Long term forecast:** One estimate puts the stock price at $430.50 in 2030.

**Bearish Predictions:**

*   **Tariff Concerns:** Tariffs could hurt profit margins and advertising revenue.
*   **Economic Slowdown:** A global economic downturn could negatively impact Amazon's business.
*   **Competition:** Increased competition could weigh on the stock.
*   **Bearish Sentiment for May 2025:** One forecast indicates that Amazon stock could dip to the $157 price range and if it fails to hold on to its resistance level, it could also dip to the $141 range.
*   **Bearish Sentiment for 2027:** Generally speaking, Amazon price prediction for 2027 is bearish. The AMZN stock is forecasted to hit a high point of $163.73 in December and reach a low of $88.67 in January.
*   **Key Factors:** Rising inflation, increased competition, or disappointing earnings reports could weigh on the stock.
*   **Long term forecast:** One estimate puts the stock price at $77 in 2030 in a bearish scenario.


# The (current) limits reached

<span style="font-size:18px;">
With two prompts Gemini 2.0 made all the effort I've spent on finance api's obsolete. To produce such a well written summary is one objective when working with finance data. This is great! Now all we need is a generative AI capable in our own language. There's a limit of course. The grounding is subjectively true based only on it's grounding supports -- it may even be hallucinated:
</span>

In [27]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

It appears you might be asking about two different companies:

**1. MGM Resorts International:**

*   **Ticker Symbol:** MGM
*   **Exchange:** New York Stock Exchange (NYSE)

**2. MGM Studios (Metro-Goldwyn-Mayer):**

*   MGM Studios is a **private** company. Therefore, it does not have a stock ticker symbol because its shares are not publicly traded on an exchange.


<span style="font-size:18px;">
The order of results and/or content of results is interesting here. The AI is confused about which MGM Studios I'm referring to. On non-thinking variants Gemini may not even mention Amazon. Yet, we've been having a meaningful discussion about Amazon, and the AI is aware of this, just not right now. Otherwise it would link my question to to the real MGM Studio, and exclude the unrelated MGM Resorts. The confusion is linked to the use of the MGM word token. The unrelated MGM stock ticker has now entered the discussion. Depending on how you prompt Gemini 2.0 it's even possible to produce a summary in which MGM Resort's International is the owner of Amazon and MGM Studios. There's two more caveat. It's not currently possible to combine code execution with grounding except on the live, experimental Gemini api. Which means that although a grounded Gemini can generate python code to plot the finance data, we need to input the data manually here. That includes matching a schema or prompting it's output.
</span>

In [28]:
response = chat.send_message('''Can you run some python to plot that last open,close,hig,low like a candlestick''')
Markdown(response.text)

I can't directly create a visual plot for you within this text-based environment. However, I can provide you with the Python code using libraries like `yfinance` and `matplotlib` that you can run in a suitable environment (like a Jupyter Notebook, Google Colab, or a local Python installation) to generate the candlestick chart.

Here's the code:

```python
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mplfinance.original_flavor import candlestick_ohlc

def plot_candlestick(ticker, start_date, end_date):
    """
    Fetches stock data and plots a candlestick chart.

    Args:
        ticker (str): The stock ticker symbol (e.g., "AMZN").
        start_date (str): The start date for the data (YYYY-MM-DD).
        end_date (str): The end date for the data (YYYY-MM-DD).
    """
    try:
        # Download stock data from Yahoo Finance
        data = yf.download(ticker, start=start_date, end=end_date)

        if data.empty:
            print(f"No data found for ticker {ticker} between {start_date} and {end_date}")
            return

        # Prepare data for candlestick chart
        data = data[['Open', 'High', 'Low', 'Close']]
        data.reset_index(inplace=True)
        data['Date'] = mdates.date2num(data['Date'].dt.date)

        # Create the candlestick chart
        fig, ax = plt.subplots()
        candlestick_ohlc(ax, data[['Date', 'Open', 'High', 'Low', 'Close']].values, width=0.6, colorup='green', colordown='red')

        # Format the chart
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.xticks(rotation=45)
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.title(f'{ticker} Candlestick Chart ({start_date} - {end_date})')
        plt.grid(True)
        plt.tight_layout()

        # Show the chart
        plt.show()

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
ticker_symbol = "AMZN"  # Or "MGM" if you want to try that one
start_date = "2025-04-01" # Adjust as needed
end_date = "2025-04-29"   # Today's date
plot_candlestick(ticker_symbol, start_date, end_date)
```

**How to use this code:**

1.  **Install Libraries:**  If you don't have them already, install the necessary libraries:

    ```bash
    pip install yfinance matplotlib mplfinance
    ```

2.  **Run the Code:** Copy and paste the code into a Python environment (like a Jupyter Notebook or a Python script) and run it.

3.  **Adjust Parameters:**
    *   `ticker_symbol`: Change this to the stock you want to analyze (e.g., "AMZN", "MGM").
    *   `start_date` and `end_date`:  Modify these to specify the date range you're interested in.  Make sure the dates are in "YYYY-MM-DD" format.

The code will download the historical stock data from Yahoo Finance and then generate a candlestick chart showing the open, high, low, and closing prices for each day within the specified date range.


In [29]:
response = chat.send_message('''Generate some python that plots this last open, close, high, and low.''')
Markdown(response.text)

Okay, I will generate Python code to plot the last available Open, Close, High, and Low values for a given stock as a candlestick. Since I don't have real-time access to the stock market, I'll create a *simplified* example that uses placeholder data.  This will demonstrate the basic plotting logic.  To use it with real data, you'd replace the placeholder data with actual stock data fetched using `yfinance` or a similar library (as in the previous example).

```python
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mplfinance.original_flavor import candlestick_ohlc
import datetime

def plot_last_candlestick(open_price, high_price, low_price, close_price, ticker_symbol="Stock"):
    """
    Plots a candlestick chart for a single day's OHLC data.

    Args:
        open_price (float): The opening price.
        high_price (float): The highest price.
        low_price (float): The lowest price.
        close_price (float): The closing price.
        ticker_symbol (str, optional): The stock ticker symbol for the chart title. Defaults to "Stock".
    """

    # Data for the candlestick
    data = [
        [mdates.date2num(datetime.date.today()), open_price, high_price, low_price, close_price]
    ]

    # Create the chart
    fig, ax = plt.subplots()
    candlestick_ohlc(ax, data, width=0.6, colorup='green', colordown='red')

    # Format the chart
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title(f'Last Day - {ticker_symbol}')
    plt.grid(True)
    plt.tight_layout()

    # Show the chart
    plt.show()


# Example Usage (replace with your actual data)
open_price = 185.00
high_price = 188.50
low_price = 184.25
close_price = 187.39
ticker = "AMZN"

plot_last_candlestick(open_price, high_price, low_price, close_price, ticker)
```

Key improvements and explanations:

*   **Clear Function Definition:**  The code is now encapsulated in a function `plot_last_candlestick` for better organization and reusability.  It takes the OHLC values as arguments.
*   **Single-Day Data:**  The code is specifically designed to plot *one* candlestick representing the last day's data.
*   **`datetime.date.today()`:**  Uses the current date for the x-axis, making the chart more informative.  The `mdates.date2num` function converts the date to a numerical format that `matplotlib` can use.
*   **Example Usage:**  The example usage section is included to show how to call the function with sample data.  **You MUST replace this sample data with your actual stock data.**
*   **Ticker Symbol:**  The `ticker_symbol` is now a parameter, allowing you to easily set the chart title.
*   **Error Handling (Removed):**  I removed the `try...except` block because, in this simplified version, there's less that can go wrong.  If you're using real data from `yfinance`, you should add error handling back in.
*   **Conciseness:** The code is streamlined to focus on the core task of plotting the candlestick.

To use this code:

1.  **Install Libraries:** Make sure you have `matplotlib` and `mplfinance` installed (`pip install matplotlib mplfinance`).
2.  **Replace Placeholder Data:**  **Crucially, replace the `open_price`, `high_price`, `low_price`, `close_price`, and `ticker` variables with the *actual* last day's OHLC data for the stock you're interested in.**  You would typically get this data from a financial API like `yfinance`.
3.  **Run the Code:** Execute the Python script.  A candlestick chart representing the last day's trading activity will be displayed.


In [30]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

Based on the search results, here's the AMZN (Amazon) data for April 28, 2025:

*   **Open:** \$190.105
*   **High:** \$190.22
*   **Low:** \$184.885
*   **Close:** \$187.70

Please note that this information is from the previous day of trading (April 28, 2025), as today is April 29, 2025.


In [31]:
response = chat.send_message(
    '''What is AMZN open,close,high,low data for the past month? 
    Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

I'm unable to retrieve and perfectly format the *complete* historical data for AMZN for the past month (April 2025) in a multi-column markdown table directly. The search results provide snippets and summaries, but not a clean, readily-copyable dataset.

However, I can give you a *partial* dataset based on the information I *could* extract, and show you the proper markdown table format.  You would then need to fill in the missing data from a reliable source like Yahoo Finance, Nasdaq, or your brokerage platform.

Here's the structure and some sample data:

```
| Date       | Open      | High      | Low       | Close     |
|------------|-----------|-----------|-----------|-----------|
| 2025-04-29 |  *N/A*    |  *N/A*    |   *N/A*   |   *N/A*   |
| 2025-04-28 | $190.105  | $190.22   | $184.885  | $187.70   |
| 2025-04-25 | $187.62   | $189.94   | $185.49   | $188.99   |
| 2025-04-24 | $180.915  | $186.74   | $180.18   | $186.54   |
| 2025-04-23 | $183.45   | $187.38   | $180.19   | $180.60   |
| 2025-04-22 | $169.845  | $176.78   | $169.35   | $173.18   |
| 2025-04-21 | $169.60   | $169.60   | $165.285  | $167.32   |
| 2025-04-17 | $176.00   | $176.21   | $172.00   | $172.61   |
| 2025-04-16 | $176.29   | $179.1046 | $171.41   | $174.33   |
| 2025-04-15 | $181.41   | $182.35   | $177.9331 | $179.59   |
| 2025-04-14 | $186.84   | $187.44   | $179.23   | $182.12   |
| 2025-04-11 | $179.93   | $185.86   | $178.00   | $184.87   |
| 2025-04-10 | $185.44   | $186.87   | $175.85   | $181.22   |
| 2025-04-09 | $172.12   | $192.65   | $169.93   | $191.10   |
| 2025-04-08 | $185.23   | $185.90   | $168.57   | $170.66   |
| 2025-04-07 | $162.00   | $183.41   | $161.38   | $175.26   |
| 2025-04-04 | $167.15   | $178.14   | $166.00   | $171.00   |
| 2025-04-03 | $183.00   | $184.13   | $176.92   | $178.41   |
| 2025-04-02 | $187.66   | $198.34   | $187.66   | $196.01   |
| 2025-04-01 | $187.86   | $193.93   | $187.20   | $192.17   |

```

**Important:**

*   **N/A:** I've marked today's data (April 29) as "N/A" because the trading day isn't over yet. You'll need to update this *after* the market closes.
*   **Verify and Complete:**  Double-check all the values and fill in any missing data points from a reliable financial source.  There might be slight discrepancies between different data providers.
*   **Markdown Table:** The code above is in Markdown format. When rendered, it will display as a table.


<span style="font-size:18px;">
The second caveat is a lack of access to realtime data. Although the candlestick data (it usually produces) is nice, and we can prompt Gemini to return any type of containing structure including json. It also produces non-deterministic output for all stock symbols. Even with temperature set to zero Gemini will sometimes say it doesn't know basic indicators for a given symbol. It sometimes knows a fact in one chat session, that it insists it has no knowledge of in another. Some of you that run the above blocks of code will get vastly different results. Sometimes including the whole month of candlestick data.
</span>

# Enter StockChat

<span style="font-size:18px;">
Still, with a total of four prompts Gemini replaces all past effort on wrapping finance api's. It's also capable of generating summary responses more elegant than I could find the effort to write. Enter StockChat, the assistant that knows finance data. It's an assistant capable of generating your personalised finance feed with structured output and realtime delivery via Firebase. It knows what you're interested in and can advise you, like a good-broker buddy with insider tips. It has the spreadsheets but knows you don't want to see them. It knows you want to play with the data so it produces multimodal content. 
<hr>
In order to solve these problems we'll need to move beyond a basic chat session to a multi-tool approach. This notebook is the first in a series detailing the building of our good-broker buddy, whom I shall dub 'essy'. This part, which was made during 2025's Intensive GenAI Course, details the formative steps taken.
</span> 

<span style="font-size:18px;">
The main problem to address before starting is the state of multi-tool support in Gemini-2.0. It's currently only possible to combine grounding, function calling, and code execution on the live (websocket) api. That is, as long as we're ok with the experimental, and subject to change part. Clearly that's not an option for our Essy. We'll start with a multi-model approach. Each expert can be good at different parts of the problem. One such expert will use function calling to chain the models together. One expert to rule them all. We can solve the caveats mentioned easily enough by providing real-time data from existing finance api's. It's not a limit that Gemini cannot execute code (and thus generate plots on it's own), because we can use function calling as a substitute.
</span>

<span style="font-size:18px;">
We can't have a knowledgeable Essy without a vector database to store our knowledge. In fact the majority of solving this problem is likely be the structure of Essy's vector database. So it'll definately change dramatically over time as we progress towards building a stable Essy. We'll use the popular Chroma and build a RAG expert to begin. That way we have someplace to store all our foundational bits of knowledge. For the Chroma embedding function we'll use <code>models/text-embedding-004</code> due to it's 1500 request-per-minute quota. We'll need to be mindful of the smaller 2,048 token input. Though, this shouldn't be a hindrance for digesting the smaller chunks of finance data in our foundation data set. For the augmented generation phase we'll use <code>models/gemini-2.0-flash</code> variants due to it's 1500 request-per-day quota.
</span>

In [32]:
# An embedding function based on text-embedding-004.
class GeminiEmbeddingFunction:
    document_mode = True # Generate embeddings for documents (T), or queries (F).

    def __init__(self, genai_client):
        self.client = genai_client
    
    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        return self.embed_with_retry(input)

    # This should catch any errors that the retry helper misses.
    def embed_with_retry(self, input: Documents, max_retries=10, delay=3) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"
            
        for attempt in range(max_retries):
            try:
                response = self.client.models.embed_content(
                    model="models/text-embedding-004",
                    contents=input,
                    config=types.EmbedContentConfig(
                        task_type=embedding_task,
                    )
                )
                return [e.values for e in response.embeddings]
            except Exception:
                time.sleep(delay * (attempt + 1))
        return None

In [33]:
# An implementation of Retrieval-Augmented Generation.
# - using Chroma and text-embedding-004 for storage and retrieval
# - using gemini-2.0-flash for augmented generation
class RetrievalAugmentedGenerator:
    chroma_client = chromadb.PersistentClient(path="vector_db")
    config_temp = types.GenerateContentConfig(temperature=0.0)

    def __init__(self, genai_client, collection_name):
        self.client = genai_client
        self.embed_fn = GeminiEmbeddingFunction(genai_client)
        self.db = self.chroma_client.get_or_create_collection(
            name=collection_name, 
            embedding_function=self.embed_fn, 
            metadata={"hnsw:space": "cosine"})

    def add_documents_list(self, docs: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        # There are separate limits on batch-embeddings and single-embeddings.
        # Rather than retry a batch-embedding, save batch-embedding for RAG-use.
        # Convert the batch document embedding to a series.
        # Note this is only for free-tier use to avoid hitting the batch-embedding quota per minute.
        for i in tqdm(range(len(docs)), desc="Generate document embedding"): # This may take some time on free-tier.
            self.db.add(ids=str(i), 
                        documents=docs[i].page_content, 
                        metadatas={"source": docs[i].metadata["source"]})

    def add_api_document(self, query: str, api_response: str, topic: str, source: str = "add_api_document"):
        self.embed_fn.document_mode = True # Switch to document mode.
        splitter = RecursiveJsonSplitter(max_chunk_size=2000) # chunk by token limit of models/text-embedding-004
        docs = splitter.create_documents(texts=[api_response], convert_lists=True)
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        # There are separate limits on batch-embeddings and single-embeddings.
        # Rather than retry a batch-embedding, save batch-embedding for RAG-use.
        # Convert the batch document embedding to a series.
        # Note this is only for free-tier use to avoid hitting the batch-embedding quota per minute.
        for i in tqdm(range(len(docs)), desc="Generate api embedding"):
            document = [{"question": query, "answer": docs[i].page_content}]
            self.db.add(ids=ids[i], 
                        documents=json.dumps(document), 
                        metadatas=[{"source": source,  "topic": topic}])

    def add_peers_document(self, query: str, peers: str, topic: str, source: str, group: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": peers}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": source,  "topic": topic, "group": group}]), 
             desc="Generate api embedding")

    def get_peers_document(self, query: str, topic: str, group: str):
        return self.get_documents_list(query, where={"$and": [{"group" : group}, {"topic": topic}]})

    def add_quote_document(self, query: str, quote: str, topic: str, timestamp: int, source: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": quote}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": source,  "topic": topic, "timestamp": timestamp}]), 
             desc="Generate api embedding")

    def get_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.get_documents_list(query, where={"$and": [{"source" : source}, {"topic": topic}]})

    def query_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.generate_answer(query, where={"$and": [{"source" : source}, {"topic": topic}]})

    def add_grounded_document(self, query: str, topic: str, result):
        self.embed_fn.document_mode = True # Switch to document mode.
        chunks = result.candidates[0].grounding_metadata.grounding_chunks
        supports = result.candidates[0].grounding_metadata.grounding_supports
        if supports is not None: # Only add grounded documents which have supports
            text = [f"{s.segment.text}" for s in supports]
            source = [f"{c.web.title}" for c in chunks]
            score = [f"{s.confidence_scores}" for s in supports]
            document = [{"text": ", ".join(text)}]
            tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": ", ".join(source), 
                                         "confidence_score": ", ".join(score), 
                                         "topic": topic,
                                         "question": query}]), 
                 desc="Generate grounding embedding")

    def get_grounding_documents(self, query: str, topic: str):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.db.get(where={"$and": [{"question" : query}, {"topic": topic}]})
            
    def add_wiki_documents(self, title: str, documents: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        result = self.get_wiki_documents(title)
        if len(result["documents"]) == 0:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(documents))))
            metas=[{"title": title, "source": "add_wiki_documents"}]*len(documents)
            # There are separate limits on batch-embeddings and single-embeddings.
            # Rather than retry a batch-embedding, save batch-embedding for RAG-use.
            # Convert the batch wiki document embedding to a series.
            # Note this is only for free-tier use to avoid hitting the batch-embedding quota per minute.
            for i in tqdm(range(len(documents)), desc="Generate wiki embeddings"):
                self.db.add(ids=ids[i], documents=documents[i], metadatas=metas[i])

    def query_wiki_documents(self, query: str, title: str):
        return self.generate_answer(query, where={"title": title})
    
    def get_wiki_documents(self, title: Optional[str] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        if title is None:
            return self.db.get(where={"source": "add_wiki_document"})
        else:
            return self.db.get(where={"title": title})

    def get_documents_list(self, query: str, max_sources: int = 10, where: Optional[dict] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        result = self.db.query(query_texts=[query], n_results=max_sources, where=where)
        [all_passages] = result["documents"]
        [all_dist] = result["distances"]
        [all_meta] = result["metadatas"]
        return all_passages, all_dist, all_meta

    def get_exchanges_csv(self, query: str):
        return self.generate_answer(query, max_sources=100, where={"source": "exchanges.csv"})

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_answer(self, query: str, max_sources: int = 1, where: Optional[dict] = None):
        all_passages, all_dist, all_meta = self.get_documents_list(query, max_sources, where)
        query_oneline = query.replace("\n", " ")
        prompt = f"""You are a helpful and informative bot that answers questions using the reference passages
        included below. Never mention the passages in your answers. Be sure to respond in concise sentences. 
        Include all relevant background information when possible. If a passage is not relevant to the answer 
        you must ignore it. If no passage answers the question respond with: I don't know.
        
        QUESTION: {query_oneline}
        """
        
        # Add the retrieved documents to the prompt.
        for passage in all_passages:
            passage_oneline = passage.replace("\n", " ")
            prompt += f"PASSAGE: {passage_oneline}\n"
    
        return self.client.models.generate_content(model=project_model, 
                                                   config=self.config_temp, 
                                                   contents=prompt)

In [34]:
# An implementation of Wiki-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by similarity to topic
# - retrieve existing groundings by similarity to topic
class WikiGroundingGenerator:
    config_temp = types.GenerateContentConfig(temperature=0.0)
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl
        with warnings.catch_warnings():
            warnings.simplefilter("ignore") # suppress beta-warning
            self.splitter = HTMLSemanticPreservingSplitter(
                headers_to_split_on=[("h2", "Main Topic"), ("h3", "Sub Topic")],
                separators=["\n\n", "\n", ". ", "! ", "? "],
                max_chunk_size=2000, # chunk by token limit of models/text-embedding-004
                chunk_overlap=50,
                preserve_links=True,
                preserve_images=True,
                preserve_videos=True,
                preserve_audio=True,
                elements_to_preserve=["table", "ul", "ol", "code"],
                denylist_tags=["script", "style", "head"],
                custom_handlers={"code": self.code_handler},
            )

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_answer(self, query: str, topic: str):
        result = self.rag.get_wiki_documents(topic)
        if len(result["documents"]) > 0:
            return self.rag.query_wiki_documents(query, topic).text
        else:
            pages = wikipedia.search(topic + " company")
            if len(pages) > 0:
                p_topic_match = 0.80
                for i in range(len(pages)):
                    if tqdm(self.get_topic_similarity(topic, pages[i]) > p_topic_match, 
                            desc= "Score wiki search by similarity to topic"):
                        request = requests.get(f"https://en.wikipedia.org/wiki/{pages[i]}")
                        documents = [document.page_content for document in self.splitter.split_text(request.text)]
                        self.rag.add_wiki_documents(topic, documents)
                        response = self.client.models.generate_content(
                            model=project_model,
                            config=self.config_temp,
                            contents=f"""You're an expert writer. You understand how to interpret html and markdown.
                                         Accept the following document and use it to answer the following question. 
                                         Don't mention the document, just answer the question. If an answer is not 
                                         possible respond with: I don't know.
                
                                         QUESTION:
                                         {query}?
                                         
                                         DOCUMENTS:
                                         {documents}""")
                        return response.text

    def code_handler(self, element: Tag) -> str:
        data_lang = element.get("data-lang")
        code_format = f"<code:{data_lang}>{element.get_text()}</code>"
        return code_format

    @retry.Retry(predicate=is_retriable)
    def get_topic_similarity(self, topic: str, page: str):
        return self.similarity_with_retry(topic, page)

    # This should catch any errors that the retry helper misses.
    def similarity_with_retry(self, topic: str, page: str, max_retries=10, delay=3):
        content = [topic + " company", page]
        for attempt in range(max_retries):
            try:
                similarity = client.models.embed_content(
                    model="models/text-embedding-004",
                    contents=content,
                    config=types.EmbedContentConfig(task_type="semantic_similarity"))
                df = pandas.DataFrame([e.values for e in similarity.embeddings], index=content)
                score = df @ df.T
                return score.iloc[0].iloc[1]
            except Exception:
                time.sleep(delay * (attempt + 1))
        return None

In [35]:
# An implementation of Grounding Generation.
# - using gemini-2.0-flash with GoogleSearch tool for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by exact match to topic
# - retrieve existing groundings by similarity to topic
class GroundingGenerator:
    config_ground = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.0
    )
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def generate_answer(self, query: str, topic: str):
        docs = self.rag.get_grounding_documents(query, topic)
        if len(docs["documents"]) > 0:
            for i in range(len(docs["metadatas"])):
                doc = docs["documents"][i]
                meta_q = docs["metadatas"][i]["question"]
                p_ground_match = 0.95 # This can be really high ~ 95-97%
                if tqdm(self.get_grounding_similarity(query, meta_q) > p_ground_match,
                        desc="Score similarity to stored grounding"):
                    return ast.literal_eval(doc)[0]["text"]
        return self.get_grounding(query, topic)

    @retry.Retry(predicate=is_retriable)
    def get_grounding_similarity(self, question: str, compare: str):
        return self.similarity_with_retry(question, compare)

    # This should catch any errors that the retry helper misses.
    def similarity_with_retry(self, question: str, compare: str, max_retries=10, delay=3):
        content = [question, compare]
        for attempt in range(max_retries):
            try:
                similarity = client.models.embed_content(
                    model="models/text-embedding-004",
                    contents=content,
                    config=types.EmbedContentConfig(task_type="semantic_similarity"))
                df = pandas.DataFrame([e.values for e in similarity.embeddings], index=content)
                score = df @ df.T
                return score.iloc[0].iloc[1]
            except Exception:
                time.sleep(delay * (attempt + 1))
        return None

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_grounding(self, query: str, topic: str):
        contents = [types.Content(role="user", parts=[types.Part(text=query)])]
        contents += f"""
        You're a search assistant that provides grounded answers to questions about {topic}. You will provide only 
        results that discuss {topic}. Be brief and specific in answering and omit extra details.
        If an answer is not possible respond with: I don't know."""
        response = self.client.models.generate_content(
            model=project_model, 
            config=self.config_ground, 
            contents=contents)
        if response.candidates[0].grounding_metadata.grounding_supports is not None:
            if topic.replace("'", "") not in response.text: # Exact topic match required
                return "I don't know." # Workaround a bug in gemini-2.0-flash (MGM Studio becomes MGM Resorts)
            else:
                self.rag.add_grounded_document(query, topic, response)
                return response.text
        return "I don't know." # Empty grounding_supports means grounding not possible for query.

# Testing the RAG Implementation

<span style="font-size:18px;">
Let's load some test data and see what the RAG can do. The test data is a CSV file containing stock market exchange data. It includes the market id code, name, locale, and operating hours. The import will use CSVLoader from <code>langchain-community</code> to parse the exchange data into Documents that our RAG can ingest.
</span>

In [36]:
# Load the exchange data from source csv.
# - Identifies exchanges by a 1-2 letter code which can be used to filter response data.
# - Also maps the exchange code to exchange details.
df = pandas.read_csv("/kaggle/input/exchanges/exchanges_src.csv").drop(["close_date"], axis=1).fillna("")
df.to_csv("exchanges.csv", index=False)
exchanges = CSVLoader(file_path="exchanges.csv", encoding="utf-8", csv_args={"delimiter": ","}).load()

# Prepare a RAG tool for use and add the exchange data.
tool_rag = RetrievalAugmentedGenerator(client, "finance")
tool_rag.add_documents_list(exchanges)

# Prepare a the grounding tools for use.
tool_wiki = WikiGroundingGenerator(client, tool_rag)
tool_ground = GroundingGenerator(client, tool_rag)

Generate document embedding: 100%|██████████| 77/77 [00:14<00:00,  5.38it/s]


<span style="font-size:18px;">
Now that the data is loaded lets ask our RAG to perform some augmenting. We can ask it to perform all sorts of useful tasks. We'll generate some useful reusable data structures and check to make sure it can answer important questions. The exchanges all have id's which are used to filter the realtime data. So we'll make sure the RAG know how to create this mapping. We'll also check it's awareness of operating hours. After all, Essy, doesn't mindlessly hammer away at api's when no new data is available.
</span>

In [37]:
# The RAG tool is a helpful expert.

response = tool_rag.get_exchanges_csv("""Give me a dictionary in string form. It must contain key:value pairs mapping 
                                         exchange code to name. Just the dictionary string in pretty form.""")
print(response.text)

response = tool_rag.get_exchanges_csv("""What is the Germany exchange code? Return only the exchange codes as a simple
                                         comma separated value that I can copy.""")
print(response.text)

response = tool_rag.get_exchanges_csv("What are the Germany exchanges and thier corresponding exchange codes?")
print(response.text, "\n")

response = tool_rag.generate_answer("What are Google's stock ticker symbols?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv("What are the US exchange operating hours?")
print(response.text, "\n")

est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
response = tool_rag.get_exchanges_csv(
    f"""Answer based on your knowledge of exchange operating hours.
        Do not answer in full sentences. Omit all chat and provide the answer only.
        All exchanges are open on weekdays. Weekdays are: Mon, Tue, Wed, Thu, Fri.
        Exchanges open and close on weekdays.
        
        The current date and time is: {datetime.now(est).strftime('%c')}
        
        When was the US exchange's last operating hours? Exclude weekends.
        Provide just the close. Include post-market hours.
        Answer with a date that uses this format: '%a %b %d %X %Y'.""")
print(response.text)

```
{
    "SC": "BOERSE_FRANKFURT_ZERTIFIKATE",
    "SX": "DEUTSCHE BOERSE Stoxx",
    "HK": "HONG KONG EXCHANGES AND CLEARING LTD",
    "DB": "DUBAI FINANCIAL MARKET",
    "NZ": "NEW ZEALAND EXCHANGE LTD",
    "QA": "QATAR EXCHANGE",
    "KS": "KOREA EXCHANGE (STOCK MARKET)",
    "SW": "SWISS EXCHANGE",
    "DU": "BOERSE DUESSELDORF",
    "BC": "BOLSA DE VALORES DE COLOMBIA",
    "KQ": "KOREA EXCHANGE (KOSDAQ)",
    "SN": "SANTIAGO STOCK EXCHANGE",
    "SI": "SINGAPORE EXCHANGE",
    "AD": "ABU DHABI SECURITIES EXCHANGE",
    "CO": "OMX NORDIC EXCHANGE COPENHAGEN A/S",
    "L": "LONDON STOCK EXCHANGE",
    "ME": "MOSCOW EXCHANGE",
    "TO": "TORONTO STOCK EXCHANGE",
    "BD": "BUDAPEST STOCK EXCHANGE",
    "TG": "DEUTSCHE BOERSE TradeGate",
    "US": "US exchanges (NYSE, Nasdaq)",
    "TW": "TAIWAN STOCK EXCHANGE",
    "JK": "INDONESIA STOCK EXCHANGE",
    "SZ": "SHENZHEN STOCK EXCHANGE",
    "VS": "NASDAQ OMX VILNIUS",
    "MX": "BOLSA MEXICANA DE VALORES (MEXICAN STOCK EXCHANGE)",
 

<span style="font-size:18px;">
Excellent! Though, despite my best effort I could not convince Gemini to apply date correction (during chaining) based on holiday. It simply wasn't stable enough to be useful. I would either have to add a holiday data set, or (what I chose) apply a quick temporary fix. A real-time API endpoint may fail due to a holiday being selected as the date. If that happens I'll just retry Thursday if the failure happened on Friday, likewise choosing Friday if the failure happened on Monday. Crude but simple for foundational purposes.
</span>

# Declaring the Function Calling Metadata

<span style="font-size:18px;">
Our Function Calling expert will chain together the other experts we've implemented thus far. It also provides the final response through augmentation. This time using the tools as a source of grounding truth. It'd like to say it's all truth organised by topic and other metadata. It's still a precarious situation if Essy incidently chains into mining data on another topic. We want Amazon to be the owner of MGM Studio's not MGM Resorts International. We also don't want a summary to include another company unless that company is a peer.
</span>

<span style="font-size:18px;">
The function calling metadata is thus extremely important. It needs to combine our other experts with the real-time api's data. Essy will use two API providers as sources of finance data. The primary motivation being that each provider has limits in their own way, yet both are useful in their own own way. This is useful anywhere you need a broad spectrum of sources of truth. At metadata creation I'll adopt the naming convention of appending the provider (if any) id. This helps keep functions more understandable when you know which provider you're dealing with.
</span>

In [38]:
# Declare callable functions using OpenAPI schema.
get_symbol_1 = types.FunctionDeclaration(
    name="get_symbol_1",
    description="""Search for the stock ticker symbol of a given company, security, isin or cusip. Each ticker
                   entry provides a description, symbol, and asset type. If this doesn't help you should try 
                   calling get_wiki_tool_response next.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The company, security, isin or cusip to search for a symbol."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["q", "exchange", "query"]
    }
)

get_name_1 = types.FunctionDeclaration(
    name="get_name_1",
    description="""Search for the name associated with a stock ticker or symbol's company, security, isin or cusip. 
    Each ticker entry provides a description, matching symbol, and asset type.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The symbol or ticker to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "company": {
                "type": "string",
                "description": "The company you're searching for."
            }
        },
        "required": ["q", "exchange", "query", "company"]
    }
)

get_symbol_quote_1 = types.FunctionDeclaration(
    name="get_symbol_quote_1",
    description="""Search for the current price or quote of a stock ticker or symbol. The response is
                   provided in json format. Each response contains the following key-value pairs:
                   
                   c: Current price,
                   d: Change,
                  dp: Percent change,
                   h: High price of the day,
                   l: Low price of the day,
                   o: Open price of the day,
                  pc: Previous close price,
                   t: Epoch timestamp of price in seconds.

                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol for a company, security, isin, or cusip." 
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "exchange": {
                "type": "string",
                "description": "The exchange code used to filter quotes. This must always be 'US'."
            }
        },
        "required": ["symbol", "query", "exchange"]
    }
)

get_local_datetime_1 = types.FunctionDeclaration(
    name="get_local_datetime_1",
    description="""Converts an array of timestamps from epoch time to the local timezone format. The result is an array
                   of date and time in locale appropriate format. Suitable for use in a locale appropriate response.
                   Treat this function as a vector function. Always prefer to batch timestamps for conversion. Use this
                   function to format date and time in your responses.""",
    parameters={
        "type": "object",
        "properties": {
            "t": {
                "type": "array",
                "description": """An array of timestamps in seconds since epoch to be converted. The order of
                                  timestamps matches the order of conversion.""",
                "items": {
                    "type": "integer"
                }
            }
        },
        "required": ["t"]
    }
)

get_market_status_1 = types.FunctionDeclaration(
    name="get_market_status_1",
    description="""Get the current market status of global exchanges. Includes whether exchanges are open or closed.  
                   Also includes holiday details if applicable. The response is provided in json format. Each response 
                   contains the following key-value pairs:

                   exchange: Exchange code,
                   timezone: Timezone of the exchange,
                    holiday: Holiday event name, or null if it's not a holiday,
                     isOpen: Whether the market is open at the moment,
                          t: Epoch timestamp of status in seconds (Eastern Time),
                    session: The market session can be 1 of the following values: 
                    
                    pre-market,regular,post-market when open, or null if closed.
                    
                    Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. The default if omitted is 'US' for the 
                                  US exchanges. A dictionary mapping supported exchange codes (key) to their 
                                  description (value) can be obtained from get_exchange_codes_1. Search the values for
                                  a matching exchange code if unsure."""
            }
        },
        "required": ["exchange"]
    }
)

get_company_peers_1 = types.FunctionDeclaration(
    name="get_company_peers_1",
    description="""Search for a company's peers. Returns a list of peers operating in the same country and in the same
                   sector, industry, or subIndustry. Each response contains the following key-value pairs: 
                   
                   symbol: The company's stock ticker symbol, 
                   peers: A list containing the peers.
                   
                   Each peers entry contains the following key-value pairs:
                   
                   symbol: The peer company's stock ticker symbol, 
                   name: The peer company's name.
                   
                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol of a company to obtain peers."
            },
            "grouping": {
                "type": "string",
                "description": """Specify the grouping category for choosing peers. When not specified the default
                                  category is subIndustry. This parameter may be one of the following values: 
                                  sector, industry, subIndustry."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "grouping", "exchange", "query"]
    }
)

get_exchange_codes_1 = types.FunctionDeclaration(
    name="get_exchange_codes_1",
    description="""Get a dictionary mapping all supported exchange codes to their names."""
)

get_exchange_code_1 = types.FunctionDeclaration(
    name="get_exchange_code_1",
    description="""Search for the exchange code to use when filtering by exchange. The result will be one or
                   more exchange codes provided as a comma-separated string value.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "Specifies which exchange code to search for."
            }
        },
        "required": ["q"]
    }
)

get_financials_1 = types.FunctionDeclaration(
    name="get_financials_1",
    description="""Get company basic financials such as margin, P/E ratio, 52-week high/low, etc. Parse the response for 
                   key-value pairs in json format and interpret their meaning as stock market financial indicators.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "metric": {
                "type": "string",
                "description": "It must always be declared as the value 'all'"
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "metric", "query"]
    }
)

get_daily_candlestick_2 = types.FunctionDeclaration(
    name="get_daily_candlestick_2",
    description="""Get a historical daily stock ticker candlestick / aggregate bar (OHLC). 
                   Includes historical daily open, high, low, and close prices. Also includes historical daily trade
                   volume and pre-market/after-hours trade prices. It does not provide today's data until after 
                   11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "date": {
                "type": "string",
                "format": "date-time",
                "description": """The date of the requested candlestick in format YYYY-MM-DD. The default is one 
                                  weekday prior to get_last_market_close (excluding weekends). This date must never 
                                  be more recent than the default. Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "date", "adjusted", "query"]
    },
)

get_company_news_1 = types.FunctionDeclaration(
    name="get_company_news_1",
    description="Retrieve the most recent news articles related to a specified ticker.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company.",
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be older than the parameter 'to'. The default
                                  value is one month ago."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be more recent than the parameter 'from'. The
                                  default value is today's date."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "from", "to", "query"]
    },
)

get_custom_candlestick_2 = types.FunctionDeclaration(
    name="get_custom_candlestick_2",
    description="""Get a historical stock ticker candlestick / aggregate bar (OHLC) over a custom date range and 
                   time interval in Eastern Time. Includes historical open, high, low, and close prices. Also 
                   includes historical daily trade volume and pre-market/after-hours trade prices. It does not
                   include today's open, high, low, or close until after 11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "multiplier": {
                "type": "integer",
                "description": "This must be 1 unless told otherwise."
            },
            "timespan": {
                "type": "string",
                "description": """The size of the candlestick's time window. This is allowed to be one of the following:
                                  second, minute, hour, day, week, month, quarter, or year. The default value is day."""
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'. The default
                                  value is one-month ago from today's date."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The 
                                  default is one weekday prior to get_last_market_close (excluding weekends).
                                  Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "sort": {
                "type": "string",
                "description": """May be one of asc or desc. asc will sort by timestmap in ascending order. desc will
                                  sort by timestamp in descending order."""
            },
            "limit": {
                "type": "integer",
                "description": """Set the number of base aggregates used to create this candlestick. This must be 5000 
                                  unless told to limit base aggregates to something else."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "multiplier", "timespan", "from", "to", "query", "adjusted", "sort", "limit"]
    },
)

get_last_market_close = types.FunctionDeclaration(
    name="get_last_market_close",
    description="""Get the date and time of the US exchange market's last close. Provides the last US market close in 
                   locale appropriate format."""
)

get_ticker_overview_2 = types.FunctionDeclaration(
    name="get_ticker_overview_2",
    description="""Retrieve comprehensive details for a single ticker symbol. It's a deep look into a company’s 
    fundamental attributes, including its primary exchange, standardized identifiers (CIK, composite FIGI, 
    share class FIGI), market capitalization, industry classification, and key dates. Also includes branding assets in
    the form of icons and logos.
    """,
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol of a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "query"]
    }
)

get_recommendation_trends_1 = types.FunctionDeclaration(
    name="get_recommendation_trends_1",
    description="""Get the latest analyst recommendation trends for a company.
                The data includes the latest recommendations as well as historical
                recommendation data for each month. The data is classified according
                to these categories: strongBuy, buy, hold, sell, and strongSell.
                The date of a recommendation indicated by the value of 'period'.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "query"]
    }
)

get_news_with_sentiment_2 = types.FunctionDeclaration(
    name="get_news_with_sentiment_2",
    description="""Retrieve the most recent news articles related to a specified ticker. Each article includes 
                   comprehensive coverage. Including a summary, publisher information, article metadata, 
                   and sentiment analysis.""",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "published_utc": {
                "type": "string",
                "format": "date-time",
                "description": """Omit this parameter unless you're told told to filter by published_utc."""
            },
            "order": {
                "type": "string",
                "description": """Must be desc for descending order, or asc for ascending order.
                                  When order is not specified the default is descending order.
                                  Ordering will be based on the parameter: sort."""
            },
            "limit": {
                "type": "integer",
                "description": """This must be 100 unless told to limit news results to something else."""
            },
            "sort": {
                "type": "string",
                "description": """The sort field used for ordering. This value must
                                  always be published_utc."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "order", "limit", "sort", "query"]
    }
)

get_rag_tool_response = types.FunctionDeclaration(
    name="get_rag_tool_response",
    description="""A database containing useful financial information. Always check here for answers first.""",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A question needing an answer. Asked as a simple string."
            }
        }
    }
)

get_wiki_tool_response = types.FunctionDeclaration(
    name="get_wiki_tool_response",
    description="""Answers questions that still have unknown answers. Retrieve a wiki page related to a company, 
                   product, or service. Each web page includes detailed company information, financial indicators, 
                   tickers, symbols, history, and products and services.""",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The question's company or product. Just the name and no other details."
            },
            "q": {
                "type": "string",
                "description": "The complete, unaltered, query string."
            }
        },
        "required": ["id", "q"]
    }
)

get_search_tool_response = types.FunctionDeclaration(
    name="get_search_tool_response",
    description="Answers questions that still have unknown answers. Use it after checking all your other tools.",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "The question needing an answer. Asked as a simple string."
            },
            "id": {
                "type": "string",
                "description": "The question's company or product. In one word. Just the name and no other details."
            }
        },
        "required": ["q", "id"]
    }
)

# Implementing the Function Calls

<span style="font-size:18px;">
One downside of this part being the main part was the lack of time to refactor this part more. Our formative Essy implements as much useful data from two finacial APIs. In order to use it you will need to declare secrets for <a class="anchor-link" href="https://finnhub.io/dashboard">Finnhub</a> and <a class="anchor-link" href="https://polygon.io/dashboard">Polygon</a> finance APIs. Register at their respective sites for your free API key. Then import the secret using the same method as how you setup Google's API key.
</span>

In [39]:
# Implement the callable functions and the function handler.

def ask_rag_tool(content):
    return tool_rag.generate_answer(content["question"], max_sources = 10).text

def ask_wiki_tool(content):
    return tool_wiki.generate_answer(content["q"], content["id"])

def ask_search_tool(content):
    return tool_ground.generate_answer(content["q"], content["id"])

def rag_exchange_codes_1(content):
    response = tool_rag.get_exchanges_csv("""Give me a dictionary in string form. It must contaihttps://api.polygon.io/v3/reference/tickers/AAPL?apiKey=4xJe226Z23RZmEc1bN8az1zz4pmNWdOpn key:value pairs 
                                             mapping exchange code to name. Just the dictionary string.
                                             Omit all other information or details. Do not chat or use sentences.""")
    codes = list(ast.literal_eval(response.text.strip("\`")).items())
    return codes

def rag_exchange_code_1(content):
    codes = tool_rag.get_exchanges_csv(f"""What is the {content} exchange code? Return only the exchange codes 
                                           as a list in string form. Just the list string.
                                           Omit all other information or details. Do not chat or use sentences.""").text
    return ast.literal_eval(codes)
    
def rag_last_market_close(content):
    est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    return tool_rag.get_exchanges_csv(
        f"""Answer based on your knowledge of exchange operating hours.
        Do not answer in full sentences. Omit all chat and provide the answer only.
        All exchanges are open on weekdays. Weekdays are: Mon, Tue, Wed, Thu, Fri.
        Exchanges open and close on weekdays.
        
        The current date and time is: {datetime.now(est).strftime('%c')}
        
        When was the US exchange's last operating hours? Exclude weekends.
        Provide just the close. Include post-market hours.
        Answer with a date that uses this format: '%a %b %d %X %Y'.""").text

def get_similarity_score(content):
    similarity = client.models.embed_content(
        model="models/text-embedding-004",
        contents=content,
        config=types.EmbedContentConfig(task_type="semantic_similarity"))
    df = pandas.DataFrame([e.values for e in similarity.embeddings], index=content)
    score = df @ df.T
    return score.iloc[0].iloc[1]
    
def impl_get_symbol_1(content, by_name: bool = True):
    response = tool_rag.get_api_documents(content["query"], content["q"], "get_symbol_1")
    if len(response[0]) == 0: # index [0] for document content
        url = f"https://finnhub.io/api/v1/search?q={content['q']}&exchange={content['exchange']}&token={FINNHUB_API_KEY}"
        try:
            response = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            matches = []
            max_failed_match = len(response["result"]) if not by_name else 3
            p_desc_match = 0.80
            p_symb_match = 0.95
            if response["count"] > 0:
                for result in tqdm(response["result"], desc="Score similarity to query"):
                    if max_failed_match > 0:
                        desc = [content['q'].upper(), result["description"].split("-", -1)[0]]
                        symb = [content['q'].upper(), result["symbol"]]
                        if by_name and get_similarity_score(desc) > p_desc_match: 
                            matches.append(result["symbol"])
                        elif not by_name and get_similarity_score(symb) > p_symb_match:
                            matches.append(result["description"])
                            max_failed_match = 0
                        else:
                            max_failed_match -= 1
            if len(matches) > 0:
                tool_rag.add_api_document(content["query"], matches, content["q"], "get_symbol_1")
                return matches
            else:
                return "I don't know."
    else:
        doc = ast.literal_eval(response[0][0])[0]
        return doc["answer"]

def impl_get_name_1(content):
    return impl_get_symbol_1(content, by_name = False)

def impl_get_quote_1(content):
    quotes = tool_rag.get_api_documents(content["query"], content["symbol"], "get_quote_1")
    isOpen = dict(impl_get_market_status_1(content))["isOpen"]
    if len(quotes[0]) == 0 or isOpen: 
        return get_current_price_1(content)
    else:
        last_close = parse(rag_last_market_close(content)).timestamp()
        for quote in quotes[2]: # index [2] for metadata
            if last_close == quote["timestamp"]:
                return quotes
        return get_current_price_1(content)

def get_current_price_1(content):
    url = f"https://finnhub.io/api/v1/quote?symbol={content['symbol']}&token={FINNHUB_API_KEY}"
    # This is a high-demand endpoint. Expect random failure under heavy (free) use.
    try:
        response = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if len(response) > 0 and response["t"] > 0:
            tool_rag.add_quote_document(content["query"], response, content["symbol"], response["t"], "get_quote_1")
            return list(response.items())
        return "I don't know."

def impl_get_market_status_1(content):
    url = f"https://finnhub.io/api/v1/stock/market-status?exchange={content['exchange']}&token={FINNHUB_API_KEY}"
    try:
        response = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if len(response) > 0:
            return list(response.items())
        return "I don't know."

def impl_get_peers_1(content):
    docs = tool_rag.get_peers_document(content["query"], content["symbol"], content['grouping'])
    if len(docs[0]) == 0: # index [0] for document content
        url = f"https://finnhub.io/api/v1/stock/peers?symbol={content['symbol']}&grouping={content['grouping']}&token={FINNHUB_API_KEY}"
        try:
            peers = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(peers) > 0:
                names = []
                for peer in peers:
                    if peer == content["symbol"]:
                        continue # skip including the query symbol in peers (included in metadata anyway)
                    name_lookup = dict(q=peer, exchange=content["exchange"], query=content["query"])
                    name = impl_get_name_1(name_lookup)
                    if name != "I don't know.":
                        p = {"symbol": peer, "name": name}
                        names.append(p)
                peers = {"symbol": content["symbol"], "peers": names}
                tool_rag.add_peers_document(content["query"], peers, content["symbol"], "get_peers_1", content['grouping'])
                return list(peers.items())
            return "I don't know."
    else:
        peers = ast.literal_eval(docs[0][0])[0]["answer"] # The first document should be most relevant.
        return list(peers.items())

def impl_local_datetime_1(content):
    local_t = []
    for timestamp in content["t"]:
        local_t.append(local_date_from_epoch(timestamp))
    return local_t

def local_date_from_epoch(timestamp):
    est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    try:
        return datetime.fromtimestamp(timestamp, tz=est).strftime('%c')
    except:
        return datetime.fromtimestamp(datetime.now(est).timestamp(), tz=est).strftime('%c')

def impl_get_financials_1(content):
    fins = tool_rag.get_api_documents(content["query"], content["symbol"], "get_financials_1")
    if len(fins[0]) == 0:
        url = f"https://finnhub.io/api/v1/stock/metric?symbol={content['symbol']}&metric={content['metric']}&token={FINNHUB_API_KEY}"
        try:
            fin = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if not fin:
                return "I don't know."
            tool_rag.add_api_document(content["query"], fin, content["symbol"], "get_financials_1")
            return list(fin.items())
    return fins

def impl_get_news_1(content):
    #news = tool_rag.get_api_documents(content["query"], content["symbol"], "get_news_1")
    #if len(news[0]) == 0:
        url = f"https://finnhub.io/api/v1/company-news?symbol={content['symbol']}&from={content['from']}&to={content['to']}&token={FINNHUB_API_KEY}"
        try:
            news = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(news) == 0:
                return "I don't know."
            #tool_rag.add_api_document(content["query"], news, content["symbol"], "get_news_1")
            return list(news.items())
    #return news

def impl_daily_candle_2(content):
    daily_candle = tool_rag.get_api_documents(content["query"], content["stocksTicker"], "daily_candle_2")
    if len(daily_candle[0]) == 0:
        url = f"https://api.polygon.io/v1/open-close/{content['stocksTicker']}/{content['date']}?adjusted={content['adjusted']}&apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            daily_candle = ast.literal_eval(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if daily_candle["status"] in ["OK","DELAYED"]:
                tool_rag.add_api_document(content["query"], daily_candle, content["stocksTicker"], "daily_candle_2")
                return list(daily_candle.items())
            else:
                date = parse(content["date"])
                new_date = None
                if date.weekday() == 4: # index 4 for friday
                    new_date = date - timedelta(days=1)
                elif date.weekday() == 0: # index 0 for monday
                    new_date = date - timedelta(days=3)
                if new_date is None:
                    return "I don't know."
                content["date"] = new_date.strftime("%Y-%m-%d")
                return impl_daily_candle_2(content)
    return daily_candle

def impl_custom_candle_2(content):
    url = f"""https://api.polygon.io/v2/aggs/ticker/{content['stocksTicker']}/range/{content['multiplier']}/{content['timespan']}/{content['from']}/{content['to']}?adjusted={content['adjusted']}&sort={content['sort']}&limit={content['limit']}&apiKey={POLYGON_API_KEY}"""
    try:
        request = requests.get(url)
        custom_candle = json.loads(request.text)
    except:
        return f"I don't know. Endpoint returned status {request.status_code}"
    else:
        if custom_candle["status"] in ["OK","DELAYED"]:
            tool_rag.add_api_document(content["query"], custom_candle, content["stocksTicker"], "custom_candle_2")
            return list(custom_candle.items())
        return "I don't know."

def impl_ticker_overview_2(content):
    overview = tool_rag.get_api_documents(content["query"], content["ticker"], "ticker_overview_2")
    if len(overview[0]) == 0:
        url = f"https://api.polygon.io/v3/reference/tickers/{content['ticker']}?apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            overview = json.loads(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if overview["status"] in ["OK","DELAYED"]:
                tool_rag.add_api_document(content["query"], overview, content["ticker"], "ticker_overview_2")
                return list(overview.items())
            return "I don't know."
    return overview

def impl_trends_1(content):
    trends = tool_rag.get_api_documents(content["query"], content["symbol"], "trends_1")
    if len(trends[0]) == 0:
        url = f"https://finnhub.io/api/v1/stock/recommendation?symbol={content['symbol']}&token={FINNHUB_API_KEY}"
        try:
            trends = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(trends) > 0:
                tool_rag.add_api_document(content["query"], trends, content["symbol"], "trends_1")
                return trends
            return "I don't know."
    return trends

def impl_get_news_2(content):
    #news = tool_rag.get_api_documents(content["query"], content["ticker"], "get_news_2")
    #if len(news[0]) == 0:
        url = f"https://api.polygon.io/v2/reference/news?ticker={content['ticker']}&order={content['order']}&limit={content['limit']}&sort={content['sort']}&apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            news = json.loads(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if news["status"] in ["OK","DELAYED"]:
                #tool_rag.add_api_document(content["query"], news, content["ticker"], "get_news_2")
                return list(news.items())
            return "I don't know."
    #return news
        
finance_tool = types.Tool(
    function_declarations=[
        get_symbol_1,
        get_name_1,
        get_symbol_quote_1,
        get_market_status_1,
        get_company_peers_1,
        get_local_datetime_1,
        get_last_market_close,
        get_exchange_codes_1,
        get_exchange_code_1,
        get_financials_1,
        get_daily_candlestick_2,
        get_custom_candlestick_2,
        get_ticker_overview_2,
        get_recommendation_trends_1,
        get_news_with_sentiment_2,
        get_rag_tool_response,
        get_wiki_tool_response,
        get_search_tool_response
    ]
)

function_handler = {
    "get_symbol_1": impl_get_symbol_1,
    "get_name_1": impl_get_name_1,
    "get_symbol_quote_1": impl_get_quote_1,
    "get_market_status_1": impl_get_market_status_1,
    "get_company_peers_1": impl_get_peers_1,
    "get_local_datetime_1": impl_local_datetime_1,
    "get_last_market_close": rag_last_market_close,
    "get_exchange_codes_1": rag_exchange_codes_1,
    "get_exchange_code_1": rag_exchange_code_1,
    "get_financials_1": impl_get_financials_1,
    "get_daily_candlestick_2": impl_daily_candle_2,
    "get_custom_candlestick_2": impl_custom_candle_2,
    "get_ticker_overview_2": impl_ticker_overview_2,
    "get_recommendation_trends_1": impl_trends_1,
    "get_news_with_sentiment_2": impl_get_news_2,
    "get_rag_tool_response": ask_rag_tool,
    "get_wiki_tool_response": ask_wiki_tool,
    "get_search_tool_response": ask_search_tool
}

In [40]:
# Define the system prompt.

instruction = f"""You are a helpful and informative bot that answers finance and stock market questions. 
Only answer the question asked and do not change topic. While the answer is still
unknown you must follow these rules for predicting function call order:

RULE#1: Always consult your other functions before get_search_tool_response.
RULE#2: Always consult get_wiki_tool_response before get_search_tool_response.
RULE#3: Always consult get_search_tool_response last.
RULE#4: Always convert timestamps with get_local_datetime_1 and use the converted date/time in your response.
RULE#5: Always incorporate as much useful information from tools and functions in your response."""

In [41]:
# Import the finance api secret keys.

POLYGON_API_KEY = UserSecretsClient().get_secret("POLYGON_API_KEY")
FINNHUB_API_KEY = UserSecretsClient().get_secret("FINNHUB_API_KEY")

In [42]:
# Implement the function calling expert.

def send_message(prompt):
    #display(Markdown("#### Prompt"))
    #print(prompt, "\n")
    # Define the user prompt part.
    contents = [types.Content(role="user", parts=[types.Part(text=prompt)])]
    # Gemini's innate notion of current date and time is unstable.
    est = pytz.timezone('US/Eastern') # The finance api data is in eastern time.
    contents += f"""
    The current date and time is: {datetime.now(est).strftime('%c')}
    
    Give a concise, and detailed summary. Use information that you learn from the API responses.
    Use your tools and function calls according to the rules. Convert any all-upper case identifiers
    to proper case in your response. Convert any abbreviated or shortened identifiers to their full forms.
    Convert timestamps according to the rules before including them.
    """
    # Enable system prompt, function calling and minimum-randomness.
    config_fncall = types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[finance_tool],
        temperature=0.0
    )
    # Handle cases with multiple chained function calls.
    function_calling_in_process = True
    while function_calling_in_process:
        # Send the user prompt and function declarations.
        response = client.models.generate_content(
            model=project_model, config=config_fncall, contents=contents
        )
        # A part can be a function call or natural language response.
        for part in response.candidates[0].content.parts:
            if function_call := part.function_call:
                # Extract the function call.
                fn_name = function_call.name
                #display(Markdown("#### Predicted function name"))
                #print(fn_name, "\n")
                # Extract the function call arguments.
                fn_args = {key: value for key, value in function_call.args.items()}
                #display(Markdown("#### Predicted function arguments"))
                #print(fn_args, "\n")
                # Call the predicted function.
                api_response = function_handler[fn_name](fn_args)[:20000] # Stay within the input token limit
                #display(Markdown("#### API response"))
                #print(api_response[:500], "...", "\n")
                # Create an API response part.
                api_response_part = types.Part.from_function_response(
                    name=fn_name,
                    response={"content": api_response},
                )
                # Append the model's function call part.
                contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) 
                # Append the api response part.
                contents.append(types.Content(role="user", parts=[api_response_part]))
            else:
                # The model gave a natural language response
                function_calling_in_process = False
                break # No more parts in response.
        if not function_calling_in_process:
            break # The function calling chain is complete.
            
    # Show the final natural language summary
    display(Markdown("#### Natural language response"))
    display(Markdown(response.text.replace("$", "\\\\$")))

# Ask a question

In [43]:
send_message("What is the current price of Amazon stock?")

Generate api embedding: 0it [00:00, ?it/s]


#### Natural language response

The current price of Amazon (AMZN) is \\$187.39. The price decreased by \\$0.31, which is a 0.1652% change. The high price of the day was \\$188.0158, and the low was \\$183.68. The market opened at \\$183.99. The previous close price was \\$187.7. The price was last updated on Tue Apr 29 16:00:00 2025.


In [45]:
send_message(
    """Tell me Amazon's current share price and provide candlestick data for the past month.
    Sort the data in descending order by date. Format the prices consistently as currency.
    Round prices to two decimal places.
    Present the data with multiple columns for display in markdown.""")

Generate api embedding: 100%|██████████| 2/2 [00:00<00:00,  5.42it/s]


#### Natural language response

As of April 29, 2025, at 21:38:55, the current price of Amazon (AMZN) is \\$187.39.

Here is the candlestick data for the past month, sorted in descending order by date:

| Date               | Open    | High    | Low     | Close   | Volume      |
| ------------------ | ------- | ------- | ------- | ------- | ----------- |
| 2025-04-28         | \\$190.11 | \\$190.22 | \\$184.89 | \\$187.70 | 33,224,732  |
| 2025-04-25         | \\$187.62 | \\$189.94 | \\$185.49 | \\$188.99 | 36,413,330  |
| 2025-04-24         | \\$180.92 | \\$186.74 | \\$180.18 | \\$186.54 | 43,051,696  |
| 2025-04-23         | \\$183.45 | \\$187.38 | \\$180.19 | \\$180.60 | 63,470,094  |
| 2025-04-22         | \\$169.85 | \\$176.78 | \\$169.35 | \\$173.18 | 56,607,202  |
| 2025-04-21         | \\$169.60 | \\$169.60 | \\$165.29 | \\$167.32 | 48,126,111  |
| 2025-04-17         | \\$176.00 | \\$176.21 | \\$172.00 | \\$172.61 | 44,726,453  |
| 2025-04-16         | \\$176.29 | \\$179.10 | \\$171.41 | \\$174.33 | 51,866,916  |
| 2025-04-15         | \\$181.41 | \\$182.35 | \\$177.93 | \\$179.59 | 43,617,902  |
| 2025-04-14         | \\$186.84 | \\$187.44 | \\$179.23 | \\$182.12 | 48,002,540  |
| 2025-04-11         | \\$179.93 | \\$185.86 | \\$178.00 | \\$184.87 | 50,594,339  |
| 2025-04-10         | \\$185.44 | \\$186.87 | \\$175.85 | \\$181.22 | 68,302,045  |
| 2025-04-09         | \\$172.12 | \\$192.65 | \\$169.93 | \\$191.10 | 116,804,328 |
| 2025-04-08         | \\$185.23 | \\$185.90 | \\$168.57 | \\$170.66 | 87,710,360  |
| 2025-04-07         | \\$162.00 | \\$183.41 | \\$161.38 | \\$175.26 | 109,297,115 |
| 2025-04-04         | \\$167.15 | \\$178.14 | \\$166.00 | \\$171.00 | 123,136,859 |
| 2025-04-03         | \\$183.00 | \\$184.13 | \\$176.92 | \\$178.41 | 95,553,617  |
| 2025-04-02         | \\$187.66 | \\$198.34 | \\$187.66 | \\$196.01 | 53,679,198  |
| 2025-04-01         | \\$187.86 | \\$193.93 | \\$187.20 | \\$192.17 | 41,246,065  |
| 2025-03-31         | \\$188.19 | \\$191.33 | \\$184.40 | \\$190.26 | 63,543,658  |

The market is currently closed.


In [46]:
send_message(
    '''Tell me about Amazon's current bullish versus bearish predictions, and recommendation trends.
    Include a discussion of any short-term trends, and sentiment analysis.''')

Generate api embedding: 100%|██████████| 1/1 [00:00<00:00,  5.05it/s]


#### Natural language response

As of April 29, 2025, here's a summary of the bullish versus bearish predictions and recommendation trends for Amazon, incorporating sentiment analysis and short-term trends from recent news:

**Recommendation Trends:**

*   Based on analyst recommendations, there's a strong bullish sentiment towards Amazon. In April 2025, there were 23 strong buy, 50 buy, and 4 hold recommendations. There were no sell or strong sell recommendations. This trend has remained relatively consistent over the past few months (January - April 2025).

**Sentiment Analysis:**

*   Recent news articles present a mixed sentiment regarding Amazon.
*   **Positive Sentiments:** Several articles highlight Amazon's strong position in e-commerce, cloud services (Amazon Web Services), and artificial intelligence as compelling reasons to invest. Some analysts predict Amazon could surpass the market valuations of Nvidia, Microsoft, and Apple within the next 5 years. The company's diversified revenue model and profitability outlook are also seen as positives. Some articles note that members of Congress have been buying Amazon stock, indicating confidence in the company's prospects.
*   **Negative Sentiments:** Concerns exist regarding the potential impact of tariffs on Amazon's business, which could lead to higher prices and reduced consumer appeal. Some articles point out that Amazon has been underperforming the S\&P 500 so far this year.
*   **Neutral Sentiments:** Some articles discuss Amazon's role in various sectors, such as travel and hospitality and the robotic software platforms market, without expressing a clear positive or negative sentiment.

**Short-Term Trends:**

*   **Tariff Concerns:** The potential impact of tariffs imposed by the Trump administration is a significant short-term concern. These tariffs could affect Amazon's e-commerce business and supply chain.
*   **AI and Cloud Growth:** Amazon's cloud computing platform (Amazon Web Services) and its investments in AI are seen as key growth drivers. The demand for AI cloud services and spending on AI infrastructure are important factors to watch.
*   **Earnings Reports:** Upcoming earnings reports from Amazon and other major tech companies are expected to cause market volatility. Investors will be looking for commentary on how tariffs could impact sales, as well as demand for AI cloud services and spending on AI infrastructure.
*   **Market Correction:** Some articles suggest that the recent stock market sell-off presents a buying opportunity for Amazon, as the stock is trading at a cheaper valuation.

**Concise Summary:**

Overall, the sentiment surrounding Amazon is cautiously optimistic. While there are concerns about the short-term impact of tariffs and the broader economic environment, analysts remain largely bullish on the company's long-term prospects, particularly due to its strong position in e-commerce, cloud computing, and AI. Recent news articles reflect this mixed sentiment, with some highlighting the company's growth potential and others raising concerns about potential headwinds.


In [47]:
send_message(
    '''Tell me about Google's share price over the past month.
    Perform a sentiment analysis of news during the same period. Include trends.''')

Generate api embedding: 100%|██████████| 2/2 [00:00<00:00,  4.66it/s]


#### Natural language response

Here's a summary of Google's (GOOG) share price and news sentiment over the past month:

**Share Price Summary:**
Over the past month, from March 31, 2025, to April 28, 2025, Google's (GOOG) share price has fluctuated.
*   The share price started at \\$154.81 on March 31, 2025.
*   It reached a high of \\$168.24 on April 25, 2025.
*   It reached a low of \\$142.66 on April 7, 2025.
*   The share price ended at \\$164.26 on April 28, 2025.

**News Sentiment Analysis:**
The news sentiment surrounding Google (GOOG) over the past month has been mixed, with a slight positive trend overall.

*   **AI Advancements:** Several articles highlight Google's (GOOG) advancements in artificial intelligence (AI), particularly in healthcare, cloud computing, and AI model development. These advancements are generally viewed positively, with analysts remaining bullish on Google's (GOOG) ability to leverage AI to drive revenue growth.
*   **Earnings and Financial Performance:** Alphabet's (GOOG) earnings reports have generally been strong, with revenue growth and expanding margins. Analysts remain optimistic about Alphabet's (GOOG) ability to leverage AI advancements and fend off competition in the search and cloud markets.
*   **Market Performance:** Some articles note that Google (GOOG) has underperformed the S&P 500 so far this year, while others suggest that the stock is undervalued and presents a buying opportunity.
*   **Partnerships and Collaborations:** Google (GOOG) has formed strategic partnerships with companies like Clover Health and Palantir, which are viewed positively.
*   **Antitrust Concerns:** Google (GOOG) continues to face antitrust lawsuits, which present a risk to the company.
*   **Tariff Concerns:** Some articles mention the potential impact of tariffs on Google's (GOOG) business, but the company's geographic diversification and strong fundamentals are expected to mitigate these challenges.
*   **Analyst Ratings:** Analysts generally remain bullish on Google's (GOOG) long-term potential, with some recommending the stock as a safe investment amid market volatility.

Overall, the news sentiment suggests that Google (GOOG) is well-positioned to benefit from the growth of AI and cloud computing, but faces some challenges from antitrust concerns and macroeconomic uncertainty.


In [50]:
send_message(
    '''How is the outlook for Apple based on trends and news sentiment over the past month?
    Perform the same analysis on Apple's peers. Then compare Apple result to it's peers.''')

#### Natural language response

The analysis of Apple (AAPL) and its peers reveals a mixed outlook based on recent trends and news sentiment.

**Apple (AAPL) Analysis:**

*   **Analyst Recommendations:** Analyst recommendations for Apple have remained relatively stable over the past month. In April 2025, there were 23 buy, 12 hold, 3 sell, 12 strong buy, and 1 strong sell recommendations. This is similar to the previous months, indicating a generally positive sentiment among analysts.
*   **News Sentiment:** Recent news articles present a mixed sentiment. Some articles highlight Apple's challenges from tariffs and a weakening Chinese economy, while others point to its strong market share in the premium hardware sector and potential upside from AI features. There are also reports of Bank of America cutting Apple's price target due to tariff risks and delayed AI rollout. However, some analysts remain bullish, citing Apple's strong brand and potential for growth.
*   **Tariffs:** The potential impact of tariffs on Apple's business is a recurring theme in the news. Some articles suggest that tariffs could hurt Apple's business, while others note that Apple has received temporary exemptions. The overall impact of tariffs remains uncertain.
*   **AI:** Apple's interest in AI features is also a recurring theme. Some articles suggest that investors should focus on Apple's interest in AI features, while others note that the company is struggling to generate excitement for its new AI features.
*   **Financial Performance:** Some articles mention that Apple is underperforming the S&P 500 so far this year, while others highlight the company's strong financial performance and potential for long-term growth.
*   **Buybacks:** Apple has been a leader in stock buybacks, which has boosted its earnings per share. However, some articles question whether the company's buyback strategy is effective given its current valuation.

**Peer Analysis:**

*   **Peers:** Apple's peers, based on the same subIndustry, include Dell Technologies (DELL), HP Inc (HPQ), Super Micro Computer Inc (SMCI), Hewlett Packard Enterprise (HPE), NetApp Inc (NTAP), Pure Storage Inc (PSTG), Western Digital Corp (WDC), and IonQ Inc (IONQ).
*   **Dell Technologies (DELL):** Dell is well-positioned to benefit from the AI surge with its expanded portfolio and Dell AI Factory. The company has consistently beaten earnings estimates, and its current stock price aligns with the bottom price target, making it an optimal entry point.
*   **HP Inc (HPQ):** HP is taking a proactive approach to addressing the digital divide and empowering nonprofit organizations to provide digital skills training and access to technology for underserved communities. This initiative aligns with their commitment to accelerate digital equity for 150 million people by 2030.

**Comparison:**

*   Apple's analyst recommendations are generally positive, but its news sentiment is mixed. Some articles highlight the company's challenges from tariffs and a weakening Chinese economy, while others point to its strong market share and potential for growth.
*   Dell Technologies is also well-positioned to benefit from the AI surge, and its stock price is attractively valued.
*   HP Inc is focused on addressing the digital divide and empowering nonprofit organizations.

**Summary:**

The outlook for Apple is cautiously optimistic. While the company faces some challenges, it also has several strengths and opportunities. The potential impact of tariffs and the success of its AI initiatives are key factors to watch. Compared to its peers, Apple's outlook is similar to Dell Technologies, which is also well-positioned to benefit from the AI surge. HP Inc is focused on addressing the digital divide, which is a different area of focus.

In conclusion, the outlook for Apple is cautiously optimistic, but investors should be aware of the potential challenges and opportunities facing the company.


In [49]:
send_message('''What does the recent news say about Apple and the impact of tariffs? Over the past 2 month.''')

#### Natural language response

Recent news regarding Apple and the impact of tariffs over the past two months (from February 29, 2025, to April 29, 2025) reveals several key points:

*   **Tariff Impact and Concerns:**
    *   Bank of America cut Apple's price target due to tariff risks and delayed artificial intelligence rollout for future iPhones, which could impact the company's long-term growth and margins.
    *   Investors are concerned about how a trade war could hurt Apple's business.
    *   A Wall Street analyst sees challenges from tariffs and a weakening Chinese economy.
    *   Apple is at the center of increasing geopolitical tensions between the U.S. and China.
    *   Apple faces potential tariffs on its devices and an economic downturn, which could hurt iPhone sales and profits.
    *   Apple stock has plunged due to tariffs on goods from China.
    *   Large tariffs on China may lead to sharply increased costs for Apple products sold in the United States, potentially leading to cratering demand and declining profits.
    *   Apple may face a 90%+ increase in iPhone costs if U.S. assembly and tariffs are enforced.
    *   Apple is shifting more iPhone production to India to mitigate the impact of tariffs, and consumers are rushing to buy iPhones before potential price hikes.
    *   Apple relies heavily on Chinese manufacturing for most of its products, placing it squarely in the crosshairs of ongoing trade tensions.
    *   Apple has received a temporary exemption from tariffs on its electronics, providing some relief.
    *   Apple is heavily reliant on manufacturing in China, but it has expanded to other countries, and the electronics industry is currently exempt from tariffs, at least temporarily.
    *   Apple is finding success in its financial services business, including Apple Pay and Apple Card, which are attracting interest from major financial institutions.
    *   Apple is actively seeking a new issuer and network operator for its Apple Card, indicating a change in its credit card strategy, but the outcome is still uncertain.
    *   Apple is working with Broadcom to develop artificial intelligence chips.

*   **Stock Performance and Analyst Ratings:**
    *   Apple has underperformed the S\&P 500 so far this year.
    *   One Wall Street analyst thinks Apple is going to \\$254.
    *   Bank of America cut Apple's price target to \\$240.
    *   Wedbush's Dan Ives remains bullish on Apple stock despite cutting the price target.
    *   Apple stock dropped 4% on Thursday as a significant individual stock decliner.
    *   Apple stock has declined 33% due to escalating trade tensions with China.
    *   Apple stock surged after the Trump administration temporarily paused reciprocal tariffs on smartphones, computers, and other electronics.
    *   Apple stock is trading at a major discount, making it an attractive investment opportunity.
    *   Apple stock is at the center of increasing geopolitical tensions between the U.S. and China, and investors will want to know how the business is coping.
    *   Apple stock has declined more than 30% from its record high four times in the last decade, but it has always rebounded sharply in the following year.

*   **Company Strategy and Developments:**
    *   Apple is shifting more iPhone production to India to mitigate the impact of tariffs.
    *   Apple is preparing for the artificial intelligence boom by designing specialized chips and launching artificial intelligence-powered features.
    *   Apple is actively seeking a new issuer and network operator for its Apple Card.
    *   Apple may be building artificial intelligence chips with Broadcom.

*   **Broader Market Trends and Comparisons:**
    *   The S\&P 500 is experiencing high concentration levels, with the top 10 stocks by market cap underperforming the index.
    *   The stock market may see significant volatility this week due to key earnings reports from major tech companies like Amazon, Apple, Meta Platforms, and Microsoft, as well as economic data releases on jobs, gross domestic product, and consumer spending.
    *   The Magnificent Seven stocks, including Apple, have underperformed the market in 2025.
    *   The Vanguard High Dividend Yield exchange traded fund is an alternative to the Magnificent Seven tech stocks that have underperformed the market in 2025.
    *   The S\&P 500 index has fallen 12.5% from its all-time high due to global trade tensions.
    *   The Vanguard S\&P 500 exchange traded fund is a potential buy during the correction.

In summary, recent news portrays a mixed outlook for Apple. While the company faces challenges from tariffs, geopolitical tensions, and slowing innovation, analysts also point to its strong brand, large installed base, and growing services revenue as reasons for optimism.


# Conclusion

<span style="font-size:18px;">
For now that will have to do. Our Essy has a solid foundation but more could be done to organise metadata. No evaluation or validation has been performed (except fuzzing the prompt). Next steps include restructuring the vector database based on lessons learned. That'll be followed by plotting, multi-modal, and structured output. The last close date (generative) function can be temperamental. In the same way Gemini always feels regarding dates. I've learnt so much. I'm happy I decided to participate in the event! It really has been a joy to see Essy grow from random chat with Gemini into the foundation for a good-broker buddy. I hope you enjoy playing with this edition as much as I enjoyed building it!
</span>